<a href="https://colab.research.google.com/github/menna161/Mining-API-Usage-Patterns/blob/main/FromCodetoPattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install astor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import ast
import astor

# Input Examples

In [3]:
ex1 = '''
x = txt.split()
print(x)
'''


ex2 ='''
for line in content.split('.'):
    print(line)
'''

ex3 = '''
words = dir.split('/')
return words[0]
'''

ex4 = '''
lines = k.split(".")
first = lines.pop()
'''

ex5 = '''
parts  = k.split('.')
line = lines.pop()
'''

ex6 = '''
parts = package_name.split('.')
name = parts.pop()
'''

# Convert Code to AST

In [4]:
tree1 = ast.parse(ex1)
tree2 = ast.parse(ex2)
tree3 = ast.parse(ex3)
tree4 = ast.parse(ex4)
tree5 = ast.parse(ex5)
tree6 = ast.parse(ex6)


dumpp1 = ast.dump(tree1)
dumpp2 = ast.dump(tree2)
dumpp3 = ast.dump(tree3)
dumpp4 = ast.dump(tree4)
dumpp5 = ast.dump(tree5)
dumpp6 = ast.dump(tree6)


print("ex1", dumpp1)
print("ex2", dumpp2)
print("ex3", dumpp3)
print("ex4", dumpp4)
print("ex5", dumpp5)
print("ex6", dumpp6)


ex1 Module(body=[Assign(targets=[Name(id='x', ctx=Store())], value=Call(func=Attribute(value=Name(id='txt', ctx=Load()), attr='split', ctx=Load()), args=[], keywords=[])), Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[]))])
ex2 Module(body=[For(target=Name(id='line', ctx=Store()), iter=Call(func=Attribute(value=Name(id='content', ctx=Load()), attr='split', ctx=Load()), args=[Str(s='.')], keywords=[]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Name(id='line', ctx=Load())], keywords=[]))], orelse=[])])
ex3 Module(body=[Assign(targets=[Name(id='words', ctx=Store())], value=Call(func=Attribute(value=Name(id='dir', ctx=Load()), attr='split', ctx=Load()), args=[Str(s='/')], keywords=[])), Return(value=Subscript(value=Name(id='words', ctx=Load()), slice=Index(value=Num(n=0)), ctx=Load()))])
ex4 Module(body=[Assign(targets=[Name(id='lines', ctx=Store())], value=Call(func=Attribute(value=Name(id='k', ctx=Load()), attr='split',

# AST To Gspan Format

In [9]:
code_snippets = [tree1, tree2, tree3, tree4, tree5, tree6]

file = open("text1.txt", "w")
i=0


# print(code_snippets)
for snippet_tree in code_snippets:
  file_write= []
  v_n = 0;
  parent_n = -1;
  file.write(f't # {i}\n')
  file.write(f'v 0 Module\n')
  i+=1
  print("tree source code")
  print(astor.to_source(snippet_tree))


  for node in ast.walk(snippet_tree):
      n = str(node).split()[0][6:]
      parent_n += 1
      children = []
      e_n = 0
      print("Node", parent_n, n)

      for x in ast.iter_child_nodes(node): 
          child = type(x).__name__
          print("childdd", child)
          name = x._fields
          print("nameee", name)
          for item,val in ast.iter_fields(x):
            if(item in ("name","id","attr")):
              child = str(child)+ "#"+item+"="+val
          v_n +=1
          # print(f'v {v_n} {child}\n')
          print("---Children",child )
          # print("list children ",list(ast.iter_child_nodes(node)))
          file_write.append('v '+str(v_n)+' '+str(child)+'\n')
          print(file_write)
          children.append([child,v_n])



      print("children ", children)
      print("list ",list(ast.iter_fields(node)))
      edges_list = list(ast.iter_fields(node))

      for edge in list(ast.iter_fields(node)):
        print("edge ", edge)
        
        if not(edge[0] in ("name","id","attr","str")) and len(children) > 0:
          # print("edge[0] ", edge[0])
          # print("edge[1] ", edge[1])

          if isinstance(edge[1], list):
            if(len(edge[1]) == 1 ):
                # print(f'e {parent_n} {children[e_n][1]} {edge[0]}\n')
                print(f'e {parent_n} {children[e_n][1]} {edge[0]}\n')
                file_write.append('e '+str(parent_n)+' '+str(children[e_n][1])+' '+str(edge[0])+'\n')
                e_n+=1
            else:
              edge_name_n = 0;
              for node in edge[1]:
                # print("node ",node)
                print(f'e {parent_n} {children[e_n][1]} {edge[0]+str(edge_name_n)}\n')
                # file.write(f'e {parent_n} {children[e_n][1]} {edge[0]+str(edge_name_n)}\n')
                file_write.append('e '+str(parent_n)+' '+str(children[e_n][1])+' '+str(edge[0]+str(edge_name_n))+'\n')
                e_n+=1
                edge_name_n+=1
          else:
            print(f'e {parent_n} {children[e_n][1]} {edge[0]}\n')
            # file.write(f'e {parent_n} {children[e_n][1]} {edge[0]}\n')
            file_write.append('e '+str(parent_n)+' '+str(children[e_n][1])+' '+str(edge[0])+'\n')
            # print("e_n ", e_n)
            e_n+=1

  print("not sorted")
  print(file_write)
  file_write.sort()
  print("sorted")
  print(file_write)
  for y in reversed(file_write):
    file.write(y)
        

file.write("t # -1\n")
file.close()



tree source code
x = txt.split()
print(x)

Node 0 Module
childdd Assign
nameee ('targets', 'value')
---Children Assign
['v 1 Assign\n']
childdd Expr
nameee ('value',)
---Children Expr
['v 1 Assign\n', 'v 2 Expr\n']
children  [['Assign', 1], ['Expr', 2]]
list  [('body', [<_ast.Assign object at 0x7fe6251c2b90>, <_ast.Expr object at 0x7fe6251c2cd0>])]
edge  ('body', [<_ast.Assign object at 0x7fe6251c2b90>, <_ast.Expr object at 0x7fe6251c2cd0>])
e 0 1 body0

e 0 2 body1

Node 1 Assign
childdd Name
nameee ('id', 'ctx')
---Children Name#id=x
['v 1 Assign\n', 'v 2 Expr\n', 'e 0 1 body0\n', 'e 0 2 body1\n', 'v 3 Name#id=x\n']
childdd Call
nameee ('func', 'args', 'keywords')
---Children Call
['v 1 Assign\n', 'v 2 Expr\n', 'e 0 1 body0\n', 'e 0 2 body1\n', 'v 3 Name#id=x\n', 'v 4 Call\n']
children  [['Name#id=x', 3], ['Call', 4]]
list  [('targets', [<_ast.Name object at 0x7fe6251c2bd0>]), ('value', <_ast.Call object at 0x7fe6251c2c10>)]
edge  ('targets', [<_ast.Name object at 0x7fe6251c2bd0>])
e